In [1]:
!pip install keplergl --quiet
import numpy as np
import pandas as pd
import geopandas as gpd
import keplergl
#from keplergl import KeplerGL
import matplotlib as plt
import plotly.express as px
#import h3
import shapely
from geopandas import GeoDataFrame,GeoSeries
import folium
import matplotlib
%matplotlib inline

In [3]:
bikelanes = pd.read_csv('bikelanes_processed.csv')
road_network = pd.read_csv('road_network_processed.csv')

In [14]:
#merging the two dataframes on the hex_cell column
merge = road_network.merge(bikelanes, on='hex_cell', how='inner')

In [21]:
merge.head()

,Unnamed: 0_x,official_street_name,road_class,geometry,latitude,longitude,coordinates_x,hex_cell,Unnamed: 0_y,installed,street_name,from_street,to_street,infra_loworder,infra_highorder,coordinates_y
0,223,WELLESLEY COTTAGES,Local / Strata,LINESTRING (-79.36711648017713 43.668687188536...,43.668863,-79.367188,"(43.66886268856369, -79.36718833021547)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"
1,56413,WELLESLEY STREET EAST,Local / Street,LINESTRING (-79.36652357999942 43.668817788586...,43.669003,-79.365687,"(43.66900293865787, -79.36568742974895)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"
2,56418,WELLESLEY STREET EAST,Local / Street,LINESTRING (-79.36711648017713 43.668687188536...,43.668752,-79.366820,"(43.668752488561395, -79.36682003008828)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"
3,225082,LANE SOUTH WELLESLEY EAST SACKVILLE STREET,Alleyway / Laneway,LINESTRING (-79.36595597979547 43.668558088567...,43.668604,-79.365733,"(43.66860392343607, -79.36573268672439)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"
4,228927,COLTSFOOT LANE,Alleyway / Laneway,LINESTRING (-79.36640367993246 43.668491588534...,43.668543,-79.366176,"(43.66854287137301, -79.36617636427408)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"


In [15]:
merge.shape

(5836, 16)

In [16]:
bikelanes.shape

(3112, 9)

In [17]:
road_network.shape

(50066, 8)

In [26]:
merge.to_csv('merge.csv', index=False)

In [ ]:
h3.